# Day 3: Parameterization of Monomers and Polymer Chain Generation

This notebook demonstrates how to parameterize monomers and generate polymer chains using **AmberTools**. It includes reusable functions to simplify the process for new molecules.

We will also visualize the molecules using **nglview**.

## Step 0: Initialization

This step sets up the environment for the tutorial. It installs required Python packages, creates necessary directories, and defines reusable functions.

**Important:** Run this step only once when starting the notebook or after restarting the kernel.

In [ ]:
# --- Lightweight "reset" for clean re-run ---
from IPython.display import clear_output
import importlib, os, sys

clear_output(wait=True)
importlib.invalidate_caches()

# Install required packages
!{sys.executable} -m pip install nglview parmed > /dev/null

import nglview as nv
import parmed as pmd

# Set fixed project root
if "PROJECT_ROOT" not in os.environ:
    os.environ["PROJECT_ROOT"] = os.path.abspath(os.getcwd())

PROJECT_ROOT = os.environ["PROJECT_ROOT"]
print("Project root:", PROJECT_ROOT)

# Create subdirectories
os.makedirs(os.path.join(PROJECT_ROOT, "monomers"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, "polymers"), exist_ok=True)

# Define reusable functions
def run_command(command):
    """Runs a shell command and prints the output."""
    print(f"Executing: {command}")
    os.system(command)

def get_monomer_dir(mol_name):
    """Returns the directory path for a given monomer."""
    monomer_dir = os.path.join(PROJECT_ROOT, "monomers", mol_name)
    os.makedirs(monomer_dir, exist_ok=True)
    return monomer_dir

def visualize_molecule(file_path):
    """
    Visualize a molecule from any supported file (PDB, MOL2, etc.) 
    with orthographic camera and fully opaque atoms (no fading).
    
    Parameters:
        file_path (str): Path to the molecular structure file.
    """
    view = nv.show_file(file_path)
     # Orthographic camera
    view.camera = 'orthographic'
    # Disable fog and make atoms fully opaque
    view.stage.set_parameters(fog=False)
    for rep in view.representations:
        rep.update_params(opacity=1.0)
    
    return view

def visualize_monomer(mol_name):
    """
    Visualize a monomer from its MOL2 file with atom indices, 
    using an orthographic camera and no fading.

    Parameters:
        mol_name (str): Name of the monomer (assumes sqm.pdb file exists in monomer directory).
    """
    mol_path = f"{get_monomer_dir(mol_name)}/sqm.pdb"
    view = nv.show_file(mol_path)
    view.add_label(radius=2, color='black',label_type='atomname')
     # Orthographic camera
    view.camera = 'orthographic'
    # Disable fog and make atoms fully opaque
    view.stage.set_parameters(fog=False)
    for rep in view.representations:
        rep.update_params(opacity=1.0)
    return view


Project root: /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer


## Step 1: Prepare Monomer from SMILES or File

This step converts a SMILES string, PDB, or XYZ file into a 3D molecular structure in MOL2 format and visualizes the result.

### Parameters:
- `mol_name`: Name of the molecule.
- `input_type`: Type of input file (`smiles`, `pdb`, or `xyz`).
- `input_data`: SMILES string or path to the input file.

In [38]:
def prepare_monomer(mol_name, input_type, input_data):
    """
    Prepares a monomer by converting SMILES or file (PDB/XYZ/MOL2) to MOL2 format.
    Warnings from OpenBabel are suppressed.

    Parameters:
        mol_name (str): Name of the molecule.
        input_type (str): 'smiles' or 'file'.
        input_data (str): SMILES string or path to input file.
    """
    monomer_dir = get_monomer_dir(mol_name)
    os.makedirs(monomer_dir, exist_ok=True)
    os.chdir(monomer_dir)
    mol2_file = f"{mol_name}.mol2"
    
    if input_type.lower() == "smiles":
        cmd = f'obabel -:"{input_data}" -O {mol2_file} --gen3d 2>/dev/null'
    elif input_type.lower() == "file":
        cmd = f'obabel {input_data} -O {mol2_file} --gen3d 2>/dev/null'
    else:
        raise ValueError("Invalid input_type. Must be 'smiles' or 'file'.")
    
    run_command(cmd)
    print(f"Monomer {mol_name} prepared successfully in {monomer_dir}.")

# Example usage: prepare PEO from SMILES
mol_name = "PEO"
prepare_monomer(mol_name, input_type="smiles", input_data="CCOCCOCCOCCOCCOCC")
visualize_molecule(f"{get_monomer_dir(mol_name)}/{mol_name}.mol2")

Executing: obabel -:"CCOCCOCCOCCOCCOCC" -O PEO.mol2 --gen3d 2>/dev/null
Monomer PEO prepared successfully in /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer/monomers/PEO.
Monomer PEO prepared successfully in /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer/monomers/PEO.


NGLWidget()

In [40]:
# Example usage: prepare PEO from SMILES
mol_name = "PET"
prepare_monomer(mol_name, input_type="file", input_data="PET_monomer.mol2")
visualize_molecule(f"{get_monomer_dir(mol_name)}/{mol_name}.mol2")

Executing: obabel PET_monomer.mol2 -O PET.mol2 --gen3d 2>/dev/null
Monomer PET prepared successfully in /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer/monomers/PET.
Monomer PET prepared successfully in /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer/monomers/PET.


NGLWidget()

## Step 2: Parameterize and Visualize Monomer

This step uses `antechamber` to assign GAFF atom types and AM1-BCC charges to the monomer.

### Parameters:
- `mol_name`: Name of the molecule.

In [31]:
def parameterize_monomer(mol_name):
    """
    Parameterizes a monomer using antechamber.
    
    Parameters:
        mol_name (str): Name of the molecule (without file extension).
    """
    monomer_dir = get_monomer_dir(mol_name)
    os.chdir(monomer_dir)
    run_command(f"antechamber -i {mol_name}.mol2 -fi mol2 -o {mol_name}_gaff.ac -fo ac -at gaff2 -an y -c bcc -nc 0 -rn {mol_name} -s 2 -pf y")
    print(f"Monomer {mol_name} parameterized successfully.")

# Parameterize the monomer
parameterize_monomer(mol_name="PEO")

#Visualize the parameterized monomer
visualize_monomer(mol_name="PEO")

Executing: antechamber -i PEO.mol2 -fi mol2 -o PEO_gaff.ac -fo ac -at gaff2 -an y -c bcc -nc 0 -rn PEO -s 2 -pf y
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff2; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for mol2 File --
   Status: pass
Info: Finished reading file (PEO.mol2); atoms read (43), bonds read (42).
Info: Determining atomic numbers from atomic symbols which are case sensitive.
Running: /Users/harish/miniconda3/envs/polymer_md/bin/bondtype -j full -i ANTECHAMBER_BOND_TYPE.AC0 -o ANTECHAMBER_BOND_TYPE.AC -f ac

Running: /Users/harish/miniconda3/envs/polymer_md/bin/atomtype -i ANTECHAMBER_AC.AC0 -o ANTECHAMBER_AC.AC -p gaff2

Running: /Users/harish/miniconda3/envs/polymer_md/bin/sqm -O -i sqm.in -o sqm.out

Running: /Users/harish/miniconda3/envs/polymer_md/bin/am1bcc -i ANTECHAMBER_AM1BCC_PRE.AC -o ANTECHAMBER_AM1BCC.AC -f ac -t bcc -p /Users/harish/mi

NGLWidget()

## Step 3: Define Chain, Head, and Tail

This step defines the chain, head, and tail of the monomer for polymerization.

### Parameters:
- `mol_name`: Name of the molecule.
- `head_id`, `tail_id`: Atom indices for the head and tail.
- `head_omit`, `tail_omit`: Atom indices to omit near the head and tail.

In [32]:
def define_chain_head_tail(mol_name, head_id, tail_id, head_omit, tail_omit, chain_id="A"):
    """
    Defines the chain, head, and tail of the monomer and saves the files.
    Also ensures a valid chain ID for tleap (replaces illegal 'X').
    
    Parameters:
        mol_name (str): Name of the molecule (without file extension).
        head_id (int): Atom index for the head.
        tail_id (int): Atom index for the tail.
        head_omit (list): Atom indices to omit near the head.
        tail_omit (list): Atom indices to omit near the tail.
        chain_id (str): Single-character chain ID (default 'A') for tleap.
    """
    monomer_dir = get_monomer_dir(mol_name)
    os.chdir(monomer_dir)
    chain_file = f"{mol_name}.chain"
    head_file = f"{mol_name}.head"
    tail_file = f"{mol_name}.tail"

    # Write chain/head/tail definition files
    with open(chain_file, "w") as chain, open(head_file, "w") as head, open(tail_file, "w") as tail:
        # Chain file
        chain.write(f"HEAD_NAME {head_id}\n")
        chain.write(f"TAIL_NAME {tail_id}\n")
        for omit in head_omit + tail_omit:
            chain.write(f"OMIT_NAME {omit}\n")
        chain.write(f"PRE_HEAD_TYPE {tail_id}\n")
        chain.write(f"POST_TAIL_TYPE {head_id}\n")
        chain.write("CHARGE 0\n")
        
        # Head file
        head.write(f"HEAD_NAME {head_id}\n")
        head.write(f"TAIL_NAME {tail_id}\n")
        for omit in tail_omit:
            head.write(f"OMIT_NAME {omit}\n")
        head.write(f"POST_TAIL_TYPE {head_id}\n")
        head.write("CHARGE 0\n")
        
        # Tail file
        tail.write(f"HEAD_NAME {head_id}\n")
        tail.write(f"TAIL_NAME {tail_id}\n")
        for omit in head_omit:
            tail.write(f"OMIT_NAME {omit}\n")
        tail.write(f"PRE_HEAD_TYPE {tail_id}\n")
        tail.write("CHARGE 0\n")

    # Run prepgen to generate .prepi files
    run_command(f"prepgen -i {mol_name}_gaff.ac -o {mol_name}.prepi -f prepi -m {mol_name}.chain -rn {mol_name} -rf {mol_name}.res")
    run_command(f"prepgen -i {mol_name}_gaff.ac -o HPT.prepi -f prepi -m {mol_name}.head -rn HPT -rf HPT.res")
    run_command(f"prepgen -i {mol_name}_gaff.ac -o TPT.prepi -f prepi -m {mol_name}.tail -rn TPT -rf TPT.res")

    print(f"Chain, head, and tail files for {mol_name} saved successfully with chain ID '{chain_id}'.")

# Define chain, head, and tail
#define_chain_head_tail(mol_name="PEO", head_id=1, tail_id=15, head_omit=[0, 17, 18, 19], tail_omit=[16, 40, 41, 42])

define_chain_head_tail(mol_name="PEO", 
                       head_id='C1', 
                       tail_id='C10', 
                       head_omit=['C', 'H', 'H1', 'H2'], 
                       tail_omit=['C11','H23','H24','H25'])

Executing: prepgen -i PEO_gaff.ac -o PEO.prepi -f prepi -m PEO.chain -rn PEO -rf PEO.res

PRE_HEAD_TYPE is   C10
POST_TAIL_TYPE is    C1
Net charge of truncated molecule is     0.00
HEAD_ATOM      2   C1
TAIL_ATOM     16  C10
MAIN_CHAIN     1    2   C1
MAIN_CHAIN     2   16  C10
OMIT_ATOM      1    1    C
OMIT_ATOM      2   18    H
OMIT_ATOM      3   19   H1
OMIT_ATOM      4   20   H2
OMIT_ATOM      5   17  C11
OMIT_ATOM      6   41  H23
OMIT_ATOM      7   42  H24
OMIT_ATOM      8   43  H25
Number of mainchain atoms (including head and tail atom):     2
Number of omited atoms:     8Executing: prepgen -i PEO_gaff.ac -o HPT.prepi -f prepi -m PEO.head -rn HPT -rf HPT.res

POST_TAIL_TYPE is    C1
Net charge of truncated molecule is     0.00
HEAD_ATOM      2   C1
TAIL_ATOM     16  C10
MAIN_CHAIN     1    2   C1
MAIN_CHAIN     2   16  C10
OMIT_ATOM      1   17  C11
OMIT_ATOM      2   41  H23
OMIT_ATOM      3   42  H24
OMIT_ATOM      4   43  H25
Number of mainchain atoms (including head and t

In [44]:
visualize_monomer(mol_name="PEO")

NGLWidget()

## Step 4: Generate Polymer Chain

This step builds a polymer chain using the defined monomer, head, and tail.

### Parameters:
- `n_mono_repeat`: Number of monomers in a repeat unit.
- `n_mono_pol`: Total number of monomers in the polymer.

In [33]:
# Parameters
n_mono_repeat = 5
n_mono_pol = 25
mol_name = "PEO"

# Set polymer directory
polymer_dir = os.path.join(PROJECT_ROOT, "polymers", mol_name)
os.makedirs(polymer_dir, exist_ok=True)

# Generate polymer sequence
repeat = " ".join([mol_name] * (n_mono_pol // n_mono_repeat - 2))
sequence = f"HPT {repeat} TPT"
print(f"Polymer sequence: {sequence}")

# Write tleap input file
tleap_input = f"""
source leaprc.gaff2
loadamberprep {get_monomer_dir(mol_name)}/{mol_name}.prepi
loadamberprep {get_monomer_dir(mol_name)}/HPT.prepi
loadamberprep {get_monomer_dir(mol_name)}/TPT.prepi
mol = sequence {{{sequence}}}
savepdb mol {polymer_dir}/{mol_name}_{n_mono_pol}mer.pdb
saveamberparm mol {polymer_dir}/{mol_name}_{n_mono_pol}mer.prmtop {polymer_dir}/{mol_name}_{n_mono_pol}mer.inpcrd
quit
"""

with open(os.path.join(polymer_dir, f"{mol_name}_tleap.in"), "w") as f:
    f.write(tleap_input)

# Run tleap and wait for the command to finish
tleap_command = f"tleap -s -f {polymer_dir}/{mol_name}_tleap.in > {polymer_dir}/{mol_name}_tleap.out"
print(f"Running tleap: {tleap_command}")
os.system(tleap_command)
print("tleap run completed. Check leap.log for details.")
os.system(f"tail -n 10 {polymer_dir}/{mol_name}_tleap.out")

Polymer sequence: HPT PEO PEO PEO TPT
Running tleap: tleap -s -f /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer/polymers/PEO/PEO_tleap.in > /Users/harish/Documents/Work_folder/MetaChem-workshops/polymermd-workshop/Day3-gen-polymer/polymers/PEO/PEO_tleap.out
tleap run completed. Check leap.log for details.
 total 0 improper torsions applied
 0 improper torsions in old prep form
Building H-Bond parameters.
Incorporating Non-Bonded adjustments.
Not Marking per-residue atom chain types.
Marking per-residue atom chain types.
 (no restraints)
	Quit

Exiting LEaP: Errors = 0; Warnings = 0; Notes = 0.


0

## Step 5: Convert to GROMACS Formats and Visualize Polymer

This step converts the AMBER topology and coordinate files into GROMACS-compatible formats.

In [42]:
# Convert to GROMACS formats
mol_name='PEO'
amber = pmd.load_file(
    f"{polymer_dir}/{mol_name}_{n_mono_pol}mer.prmtop", 
    f"{polymer_dir}/{mol_name}_{n_mono_pol}mer.inpcrd"
)
amber.save(f"{polymer_dir}/{mol_name}_{n_mono_pol}mer.gro", overwrite=True)
amber.save(f"{polymer_dir}/topol.top", format="gromacs", overwrite=True)

print(f"Conversion completed: .gro and .top files generated for {mol_name}.")

Conversion completed: .gro and .top files generated for PEO.


In [43]:
# Visualize the polymer chain
visualize_molecule(f"{polymer_dir}/{mol_name}_{n_mono_pol}mer.gro")

NGLWidget()